In [1]:
import pandas as pd

data = pd.read_parquet("../pipelines/data/db_pull_calibrated.parquet")
data

,timestamp,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,activity,hash,person
0,2023-03-16 15:50:20.660,0.856745,-9.736820,0.020757,0.268116,-1.078746,1.059684,-17.852205,35.951828,-0.945442,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
1,2023-03-16 15:50:20.670,0.909825,-8.220062,1.002787,0.271234,-1.028209,1.029722,-17.822435,35.816177,-1.256371,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
2,2023-03-16 15:50:20.680,0.336442,-7.226947,2.845873,0.084436,-0.911180,0.878822,-17.261415,35.596741,-1.602486,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
3,2023-03-16 15:50:20.690,-0.488674,-6.452185,5.054952,-0.314842,-0.878484,0.670179,-17.013292,35.198273,-1.386124,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
4,2023-03-16 15:50:20.700,-1.977320,-5.265067,5.953660,-0.817032,-0.921044,0.375689,-16.602587,35.248901,-1.095116,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314652,2023-03-12 13:25:03.670,8.873645,-12.125825,7.280043,-1.097083,-2.545262,-3.277552,-32.639999,26.099998,23.580000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314653,2023-03-12 13:25:03.680,3.578034,-10.812859,7.249442,-0.878516,-3.433582,-2.865095,-33.480000,24.660000,24.240000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314654,2023-03-12 13:25:03.690,-0.935012,-8.310925,6.780705,-0.698311,-3.857278,-2.475241,-33.480000,23.400000,25.379999,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314655,2023-03-12 13:25:03.700,-4.368380,-5.661316,7.183708,-0.454698,-3.775789,-2.106645,-34.380001,21.660000,26.760000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess


In [2]:
data = data[data["hash"] == "040d27bca3b84ad198a2aa1129e4c8ac"].copy()
data = data[
    [
        "timestamp",
        "Accelerometer_x",
        "Accelerometer_y",
        "Accelerometer_z",
        "Gyroscope_x",
        "Gyroscope_y",
        "Gyroscope_z",
        "Magnetometer_x",
        "Magnetometer_y",
        "Magnetometer_z",
    ]
].copy()

In [3]:
data

,timestamp,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z
0,2023-03-16 15:50:20.660,0.856745,-9.736820,0.020757,0.268116,-1.078746,1.059684,-17.852205,35.951828,-0.945442
1,2023-03-16 15:50:20.670,0.909825,-8.220062,1.002787,0.271234,-1.028209,1.029722,-17.822435,35.816177,-1.256371
2,2023-03-16 15:50:20.680,0.336442,-7.226947,2.845873,0.084436,-0.911180,0.878822,-17.261415,35.596741,-1.602486
3,2023-03-16 15:50:20.690,-0.488674,-6.452185,5.054952,-0.314842,-0.878484,0.670179,-17.013292,35.198273,-1.386124
4,2023-03-16 15:50:20.700,-1.977320,-5.265067,5.953660,-0.817032,-0.921044,0.375689,-16.602587,35.248901,-1.095116
...,...,...,...,...,...,...,...,...,...,...
6341,2023-03-16 15:51:24.070,-0.909179,-8.443210,8.876128,0.618988,-0.578190,1.235658,-24.672789,27.058319,12.749062
6342,2023-03-16 15:51:24.080,-2.368730,-7.690176,8.344416,0.238817,-0.929550,1.484196,-23.223375,27.085190,12.312843
6343,2023-03-16 15:51:24.090,-3.954403,-5.228138,4.458433,0.092740,-1.009917,1.521586,-21.532606,27.084213,11.729782
6344,2023-03-16 15:51:24.100,-4.390823,0.273040,0.455637,0.105977,-0.898056,1.425018,-19.915117,27.464844,11.317650


In [11]:
# send data via api request
import requests
import brotli
import json


method = "POST"
url = "http://127.0.0.1:8000/CNN"
payload = data.to_parquet()
payload = brotli.compress(payload)

response = requests.request(method, url, data=payload)
text = response.content.decode("utf-8")
text = json.dumps(json.loads(text), indent=2)

print(f"Response Code: {response.status_code}")
print(text)

# 12 Segments, all were correctly classified

Response Code: 200
{
  "0": "Rennen",
  "1": "Rennen",
  "2": "Rennen",
  "3": "Rennen",
  "4": "Rennen",
  "5": "Rennen",
  "6": "Rennen",
  "7": "Rennen",
  "8": "Rennen",
  "9": "Rennen",
  "10": "Rennen",
  "11": "Rennen"
}
